# VOCS v1 data structure 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [3]:
?VOCS

In [4]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [5]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [6]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [7]:
# Check that these are the same
vocs2 == vocs


True

In [8]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [9]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [10]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [11]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [12]:
from xopt.vocs import objective_data, constraint_data, feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,659.417792,0.079898,1234,659.497691,659.337894,1318.995382,1318.675788
6,345.829238,0.722414,1234,346.551653,345.106824,693.103306,690.213648
7,561.200776,0.595394,1234,561.796171,560.605382,1123.592341,1121.210763
8,374.554027,-0.317771,1234,374.236256,374.871797,748.472512,749.743595
9,640.101368,0.058249,1234,640.159617,640.043119,1280.319235,1280.086237
10,288.698495,-0.729370,1234,287.969126,289.427865,575.938251,578.855730
11,903.867566,-0.705518,1234,903.162048,904.573084,1806.324096,1809.146167
12,476.225015,-0.374257,1234,475.850758,476.599272,951.701516,953.198545
13,100.612065,-0.991945,1234,99.620120,101.604009,199.240240,203.208018
14,266.668374,-0.676580,1234,265.991794,267.344954,531.983589,534.689908


In [13]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [14]:
?objective_data

In [15]:
# These are in standard form for minimization
objective_data(vocs, data)

,objective_c,objective_d
5,-659.497691,659.337894
6,-346.551653,345.106824
7,-561.796171,560.605382
8,-374.236256,374.871797
9,-640.159617,640.043119
10,-287.969126,289.427865
11,-903.162048,904.573084
12,-475.850758,476.599272
13,-99.620120,101.604009
14,-265.991794,267.344954


In [16]:
# use the to_numpy() method to convert for low level use.
objective_data(vocs, data).to_numpy()

array([[-659.49769077,  659.33789413],
       [-346.55165275,  345.10682393],
       [-561.79617052,  560.60538174],
       [-374.23625597,  374.87179738],
       [-640.15961748,  640.04311862],
       [-287.96912563,  289.42786501],
       [-903.1620478 ,  904.57308365],
       [-475.85075798,  476.59927241],
       [ -99.6201201 ,  101.60400924],
       [-265.9917943 ,  267.34495419]])

In [17]:
vocs.constraints

{'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}

In [18]:
?constraint_data

In [19]:
constraint_data(vocs, data)

,constraint_e,constraint_f
5,1316.995382,-1318.675788
6,691.103306,-690.213648
7,1121.592341,-1121.210763
8,746.472512,-749.743595
9,1278.319235,-1280.086237
10,573.938251,-578.855730
11,1804.324096,-1809.146167
12,949.701516,-953.198545
13,197.240240,-203.208018
14,529.983589,-534.689908


In [20]:
feasibility_data(vocs, data)

,feasibility_e,feasibility_f,feasibility
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
